# Axial Drilling Maps

---
## Load All Data

In [ ]:
import pandas as pd
import xarray as xr
xr.set_options(display_style='html');

In [ ]:
data_dir = '/home/jovyan/repos/axial-drilling/maps/data/'

### Load Vent Data

In [ ]:
vents_df = pd.read_excel(data_dir + '/markers-vents-axial-master-updated-post2017.xlsx', sheet_name = 1,
                     usecols=[0, 1, 2, 7, 10], names=['name', 'lat', 'lon', 'type', 'use'])
vents_df.use = vents_df.use.str.casefold()
vents_df = vents_df[vents_df.use == 'yes'].reset_index(drop = True) # drop vents with use=='no'
vents_df.head()

### Load Bathy Data

In [ ]:
bathy_file = 'axial_auv_mbari09and11_clean_gcs_1m_v3.grd'

In [ ]:
bathy_ds = xr.load_dataset(data_dir + bathy_file)
bathy_ds = bathy_ds.rename({'x': 'lon', 'y': 'lat', 'z': 'depth'})
bathy_ds

### Load OOI Infrastructure Data

In [ ]:
ooi_df = pd.read_excel(data_dir + 'RSN_Positions_20190906_PUB.xlsx', sheet_name = 2,
                     usecols=[0, 1, 2, 8], names=['name', 'lat', 'lon', 'type'])
boundary_east = -129.8
ooi_df = ooi_df[ooi_df.lon < boundary_east].reset_index(drop = True) # remove infrastructure west of Axial base
ooi_df.head()

In [ ]:
ooi_cables_df = pd.read_excel(data_dir + 'RSN_Positions_20190906_PUB.xlsx', sheet_name = 5,
                     usecols=[1, 2, 3], names=['name', 'lat', 'lon'])
ooi_cables_df = ooi_cables_df[ooi_cables_df.lon < boundary_east].reset_index(drop = True)
ooi_cables_df.head()

### Load Layer 2A Thickness Data

In [ ]:
thickness_df = pd.read_csv(data_dir + 'l2A_3D.csv', names=['lon', 'lat', 'thickness'])
thickness_df.thickness = round(thickness_df.thickness*1000)

In [ ]:
lat = thickness_df.lat.values.reshape(301, 401).tolist()
lon = thickness_df.lon.values.reshape(301, 401).tolist()
thickness = thickness_df.thickness.values.reshape(301, 401).tolist()

In [ ]:
thickness_ds = xr.Dataset({'thickness': (['x', 'y'], thickness)},
                coords = {'lon': (['x', 'y'], lon),
                          'lat': (['x', 'y'], lat)})
thickness_ds

### Define Hole Locations

In [ ]:
holes = [['AXIAL_01A', 45.925414, -129.977862],
         ['AXIAL_02A', 45.926026, -129.971828],
         ['AXIAL_03A', 45.916465, -129.976786],
         ['AXIAL_04A', 45.920833, -129.977222]]
holes_df = pd.DataFrame(holes, columns = ['name', 'lat', 'lon']).set_index('name')
holes_df

### Define Circle Locations

In [ ]:
from cartopy import geodesic as gd

In [ ]:
def circle(lat, lon, radius):
    circ = gd.Geodesic().circle(lon=lon, lat=lat, radius=radius, n_samples=40, endpoint=True)
    return pd.DataFrame({'lon': list(circ[:,0]), 'lat': list(circ[:,1])})

In [ ]:
RS03W8_circles = []
radius = 20
for i in ooi_cables_df[ooi_cables_df.name=='RS03W8'].index:
    lat = ooi_cables_df[ooi_cables_df.name=='RS03W8'].loc[i].lat
    lon = ooi_cables_df[ooi_cables_df.name=='RS03W8'].loc[i].lon
    RS03W8_circles.append(circle(lat, lon, radius))

In [ ]:
RS03W6_circles = []
radius = 20
for i in ooi_cables_df[ooi_cables_df.name=='RS03W6'].index:
    lat = ooi_cables_df[ooi_cables_df.name=='RS03W6'].loc[i].lat
    lon = ooi_cables_df[ooi_cables_df.name=='RS03W6'].loc[i].lon
    RS03W6_circles.append(circle(lat, lon, radius))

In [ ]:
ooi_circles = []
radius = 20
for i in ooi_df.index:
    lat = ooi_df.loc[i].lat
    lon = ooi_df.loc[i].lon
    ooi_circles.append(circle(lat, lon, radius))

---
## Interactive Plots
These plots using Holoviews and Hvplot are rough and meant for exploring the data and picking hole locations. The GMT plots below will be pulication-ready and finished once we have decided on hole locations and the elements to include.

In [ ]:
import cartopy.crs as ccrs
import hvplot.pandas
import hvplot.xarray
import holoviews as hv

#### Layer 2A Thickness Overlay

In [ ]:
plot_opts = {'x': 'lon', 'y': 'lat', 'geo': True, 'hover': True, 'frame_height': 600, 'frame_width': 800}
circle_color = 'white'

In [ ]:
RS03W8_circles_hvplots = []
for RS03W8_circle in RS03W8_circles:
    RS03W8_circles_hvplots.append(RS03W8_circle.hvplot.paths(**plot_opts, c = circle_color))

In [ ]:
RS03W6_circles_hvplots = []
for RS03W6_circle in RS03W6_circles:
    RS03W6_circles_hvplots.append(RS03W6_circle.hvplot.paths(**plot_opts, c = circle_color))

In [ ]:
ooi_circles_hvplots = []
for ooi_circle in ooi_circles:
    ooi_circles_hvplots.append(ooi_circle.hvplot.paths(**plot_opts, c = circle_color))

In [ ]:
ooi_cables_hvplots = []
for name, group in ooi_cables_df.groupby('name'):
    ooi_cables_hvplots.append(group.hvplot.paths(**plot_opts, c = circle_color))

In [ ]:
thickness_ds.hvplot.quadmesh(**plot_opts, hover_cols=['lon', 'lat', 'thickness']).opts(clim=(450,700)) \
  * hv.Overlay(RS03W8_circles_hvplots + RS03W6_circles_hvplots + ooi_circles_hvplots + ooi_cables_hvplots) \
  * vents_df.hvplot.points(**plot_opts, c='red', s=10, hover_cols=['lon', 'lat', 'name']) \
  * ooi_df.hvplot.points(**plot_opts, c='black', s=10, hover_cols=['lon', 'lat', 'name']) \
  * holes_df.hvplot.points(**plot_opts, c='yellow', s=100)

#### Bathy Overlay

In [ ]:
plot_opts = {'x': 'lon', 'y': 'lat', 'geo': True, 'hover': True, 'frame_height': 600, 'frame_width': 800}

In [ ]:
bathy_ds.depth[::10].hvplot.image(**plot_opts, cmap = 'bmy').opts(clim=(-1550,-1500)) \
  * hv.Overlay(RS03W8_circles_hvplots + RS03W6_circles_hvplots + ooi_circles_hvplots + ooi_cables_hvplots) \
  * vents_df.hvplot.points(**plot_opts, c='cyan', s=10, hover_cols=['lon', 'lat', 'name']) \
  * ooi_df.hvplot.points(**plot_opts, c='black', s=10, hover_cols=['lon', 'lat', 'name']) \
  * holes_df.hvplot.points(**plot_opts, c='yellow', s=100)

---
## GMT Plots

In [ ]:
import os
import pandas as pd
import pygmt
import numpy as np

### Bathy

In [ ]:
import os
import pandas as pd
import pygmt
import numpy as np

In [ ]:
cmd = 'gmt grdgradient %s%s -Gshading.grd -A40/10 -Ne2' % (data_dir, bathy_grd)
print(cmd)
os.system(cmd)

In [ ]:
# plot bathy
# *** make NaN white ***
outfile = 'axial_auv_mbari09and11_clean_gcs_1m_v3.grd'
fig = pygmt.Figure()
fig.basemap(region=[-130-3.75/60, -129-57/60, 45+54/60, 46], projection="M-130/46/10i", frame=True)
fig.grdimage('%s%s' % (data_dir, outfile), shading='shading.grd')
#fig.show(width=400)

In [ ]:
fig_base = fig

In [ ]:
fig_base.plot(df_ooi_objects.lon.values, df_ooi_objects.lat.values, style="c0.2", color="red")
fig_base.plot(df_vents_plot.lon.values, df_vents_plot.lat.values, style="i0.3c", color="cyan")
for cable in df_ooi_cables.name.unique():
    cable_locs = df_ooi_cables.loc[df_ooi_cables.name == cable]
    fig_base.plot(df_cable_locs.lon.values, df_cable_locs.lat.values, W='1p,white')
fig_base.show(width=800)

In [ ]:
# plot bathy
# *** make NaN white ***
outfile = 'axial_auv_mbari09and11_clean_gcs_1m_v3.grd'
fig_base_zoom = pygmt.Figure()
fig_base_zoom.basemap(region=[-(130+0.5/60), -(129+58.5/60), 45+55.25/60, 45+55.5/60], projection="M-130/46/10i", frame=True)
fig_base_zoom.grdimage('%s%s' % (data_dir, outfile), shading='shading.grd')
#fig.show()

In [ ]:
fig_base_zoom_base = fig_base_zoom

In [ ]:
fig_base_zoom_base.plot(ooi_objects.lon.values, ooi_objects.lat.values, style="c0.2", color="red")
fig_base_zoom_base.plot(vents_plot.lon.values, vents_plot.lat.values, style="i0.3c", color="cyan")
for cable in ooi_cables.name.unique():
    cable_locs = ooi_cables.loc[ooi_cables.name == cable]
    fig_base_zoom_base.plot(cable_locs.lon.values, cable_locs.lat.values, W='1p,white')
fig_base_zoom_base.show(width=800)

In [ ]:
fig_base_zoom_base.plot(ooi_objects.lon.values, ooi_objects.lat.values, style="c0.2", color="red")
fig_base_zoom_base.plot(vents_plot.lon.values, vents_plot.lat.values, style="i0.3c", color="cyan")
for cable in ooi_cables.name.unique():
    cable_locs = ooi_cables.loc[ooi_cables.name == cable]
    fig_base_zoom_base.plot(cable_locs.lon.values, cable_locs.lat.values, W='1p,white')
fig_base_zoom_base.show(width=800)

### Vents

In [ ]:
fig.plot(vents_plot.lon.values, vents_plot.lat.values, style="i0.3c", color="cyan")
fig.show()

### OOI Cable Infrastructure

In [ ]:
fig.plot(ooi_objects.lon.values, ooi_objects.lat.values, style="c0.2", color="red")

In [ ]:
for cable in ooi_cables.name.unique():
    cable_locs = ooi_cables.loc[ooi_cables.name == cable]
    fig.plot(cable_locs.lon.values, cable_locs.lat.values, W='yellow')
fig.show(width=800)

---
## Scratch

In [ ]:
df_rad_300_1 = circle(45.9263, -129.9791, 300)
df_rad_300_2 = circle(45.9258, -129.9776, 300)

In [ ]:
holes = [['AXIAL_01A', 45.925414, -129.977862],
         ['AXIAL_02A', 45.926026, -129.971828],
         ['AXIAL_03A', 45.916465, -129.976786],
         ['AXIAL_04A', 45.920833, -129.977222]]
holes_df = pd.DataFrame(holes, columns = ['name', 'lat', 'lon']).set_index('name')
holes_df

In [ ]:
hole_names = ['AXIAL_01A', 'AXIAL_01B',  'AXIAL_01C',  'AXIAL_01D']
axial_01A = [45.925414, -129.977862]
axial_01B = [45.926026, -129.971828]
axial_01C = [45.916465, -129.976786]
axial_01D = [45.9208333, -129.9772222]

data = [{'lat': axial_01a[0], 'lon':axial_01a[1]}, 
        {'lat': axial_01b[0], 'lon': axial_01b[1]}, 
        {'lat': axial_01c[0], 'lon': axial_01c[1]}, 
        {'lat': axial_01d[0], 'lon': axial_01d[1]}]

df_holes = pd.DataFrame(data, index =['axial_01A', 'axial_01B',  'axial_01C',  'axial_01D'])
df_holes

In [ ]:
plot_opts('x': 'lon', 'y': 'lat', 'geo': True, 'hover': True, 'frame_height': 600, 'frame_width': 800)
ds_thickness.hvplot.quadmesh(**plot_opts, hover_cols=['lon', 'lat', 'thickness']) \
  * df_vents_plot.hvplot.points(**plot_opts, c='red', s=10, hover_cols=['lon', 'lat', 'name']) \
  * df_ooi_objects_plot.hvplot.points(**plot_opts, c='black', s=10, hover_cols=['name']) \
  * df_ooi_cables_plot.hvplot.points(**plot_opts, c = 'black', s = 2) \
  * df_rad_300_1.hvplot.paths(x = 'lon', y = 'lat', geo = True, c = 'blue') \
  * df_rad_300_2.hvplot.paths(x = 'lon', y = 'lat', geo = True, c = 'blue') \
  * df_holes.hvplot.points(x = 'lon', y = 'lat', geo = True, c = 'yellow')

In [ ]:
df_ooi_cables_plot.hvplot.line(x = 'lon', y = 'lat', geo = True, by = 'name', c = 'red', dynamic = False)

In [ ]:
ax = plt.axes(projection=ccrs.Mercator())
ds.thickness.plot(x = 'lon', y = 'lat')
#ax.stock_img()
plt.show()

In [ ]:
proj = ccrs.Mercator()
plt.figure(figsize=(30,20))
ax = plt.axes(projection=proj)

#ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
#ax.coastlines(zorder=1)
xticks = range(0, 361, 10000)
ax.set_xticks(xticks, crs=proj)
yticks = range(-90, 91, 30)
ax.set_yticks(yticks, crs=proj)
#ax.set_title('central longitude = 180')
ax.scatter(x = ds.lon, y = ds.lat, s=0.1);

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs

gv.extension('bokeh', 'matplotlib')

In [ ]:
#scatter = hv.Scatter(data=df, kdims = ['lon', 'lat'], vdims=['thickness'])
scatter = hv.Scatter(data=df, vdims = ['lon', 'lat', 'thickness'])
scatter.opts(frame_width=600, frame_height=600, colorbar=True, axiswise=True)

In [ ]:
scatter = hv.Scatter(ds, vdims=['lon', 'lat', 'thickness'])
scatter

In [ ]:
qmesh = gv.Dataset(ds.thickness).to(gv.QuadMesh)

In [ ]:
gvds = gv.Dataset(ds)
gvds

In [ ]:
type(gvds)

In [ ]:
import geoviews as gv
gv.extension('bokeh')
import holoviews as hv

In [ ]:
quadmesh.thickness.to(hv.HoloMap)

In [ ]:
quadmesh = gvds.to(gv.QuadMesh, ['lon', 'lat'])

In [ ]:
type(quadmesh)

In [ ]:
quadmesh

In [ ]:
qmesh = gv.Dataset(ds.thickness[::3,::2]).to(gv.QuadMesh, groupby='time')

In [ ]:
type(qmesh)

In [ ]:
type(rasm.Tair)

In [ ]:
type(ds.thickness)

In [ ]:
rasm.Tair

In [ ]:
ds.thickness

In [ ]:
gvds = gv.Dataset(ds, ['lon', 'lat'], 'thickness', crs=crs.Mercator())
#images = dataset.to(gv.Image)
gvds

In [ ]:
qmesh = gv.Dataset(ds.thickness).to(gv.QuadMesh)

In [ ]:
qmesh = gvds.to(gv.QuadMesh)

In [ ]:
qmesh.to(hv.HoloMap)

In [ ]:
dir(qmesh)

In [ ]:
qmesh.opts(projection=ccrs.Mercator())

In [ ]:
gmesh

In [ ]:
plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ds.thickness.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat', add_colorbar=False)
ax.coastlines()
ax.set_ylim([0,90]);

In [ ]:
ds.to_netcdf('../data/test.nc')

In [ ]:
data_dir = '/home/jovyan/github/ooicloud/axial-drilling/maps/data/'
depth_file = 'test.nc'
fig = pygmt.Figure()
fig.basemap(region=[-130-3.75/60, -129-57/60, 45+54/60, 46], projection="M-130/46/10i", frame=True)
fig.grdimage('%s%s' % (data_dir, depth_file))

In [ ]:
# create a grid that spans 0 359 E, and -89 to 90 N.
longitude = np.arange(0, 360, 1)
latitude = np.arange(-89, 91, 1)
x = np.sin(np.deg2rad(longitude))
y = np.linspace(start=0, stop=1, num=180)
data = y[:, np.newaxis] * x

# create a DataArray, and then export this as a netcdf file
dataarray = xr.DataArray(data, coords=[('latitude', latitude,
                                       {'units': 'degrees_north'}),
                                       ('longitude', longitude,
                                       {'units': 'degrees_east'})], 
                         attrs = {'actual_range': [-1, 1]})
dataset = dataarray.to_dataset(name='dataarray')
dataset.to_netcdf('test.grd')

fig = pygmt.Figure()

# create a projected image using the DataArray in memory and the netcdf file
fig.grdimage(dataset.dataarray, region='g', projection='A0/0/6i')
fig.grdimage('test.grd', region='g', projection='A0/0/6i', X='6.2i')
    

In [ ]:
fig.show()

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import geoviews as gv

gv.extension('matplotlib')
gv.output(fig='svg', size=300)

In [ ]:
qmeshes.opts(projection=ccrs.GOOGLE_MERCATOR)

In [ ]:
tiles = gv.tile_sources.EsriImagery
rasm = xr.tutorial.open_dataset('rasm').load()
qmeshes = gv.Dataset(rasm.Tair[::4, ::3, ::3]).to(gv.QuadMesh, groupby='time')

---
## Clean up GRD files
Only do once!

In [ ]:
# clip bad values
data_dir = '/home/jovyan/github/ooicloud/axial-drilling/maps/data/'

infile = 'axial_auv_mbari09and11n_gcs_1m_v3.grd'
tmpfile1 = 'tmp1.grd'
cmd = ('gmt grdclip %s%s -G%s%s -Sb-1600/NaN -Sa1500/NaN' % (data_dir, infile, data_dir, tmpfile1))
os.system(cmd);

infile = 'axial_auv_mbari09and11c_gcs_1m_v3.grd'
tmpfile2 = 'tmp2.grd'
cmd = ('gmt grdclip %s%s -G%s%s -Sb-1600/NaN -Sa1500/NaN' % (data_dir, infile, data_dir, tmpfile2))
os.system(cmd);

infile = 'axial_auv_mbari09and11s_gcs_1m_v3.grd'
tmpfile3 = 'tmp3.grd'
cmd = ('gmt grdclip %s%s -G%s%s -Sb-1600/NaN -Sa1500/NaN' % (data_dir, infile, data_dir, tmpfile3))
os.system(cmd);

In [ ]:
# blend GRD files
tmpfile4 = 'tmp4.grd'
cmd = ('gmt grdblend %s%s %s%s -G%s%s' % (data_dir, tmpfile1, data_dir, tmpfile2, data_dir, tmpfile4))
os.system(cmd);

tmpfile5 = 'tmp5.grd'
cmd = ('gmt grdblend %s%s %s%s -G%s%s' % (data_dir, tmpfile4, data_dir, tmpfile3, data_dir, tmpfile5))
os.system(cmd);

In [ ]:
# crop region
bathy_grd = 'axial_auv_mbari09and11_clean_gcs_1m_v3.grd'
cmd = ('gmt grdcut %s%s -G%s%s -R%0.8f/%0.8f/%0.8f/%0.8f' % (data_dir, tmpfile5, data_dir, bathy_grd, -130-3.75/60, -129-57/60, 45+54/60, 46))
os.system(cmd);

In [ ]:
# clean up
os.system('rm %s%s' % (data_dir, tmpfile1));
os.system('rm %s%s' % (data_dir, tmpfile2));
os.system('rm %s%s' % (data_dir, tmpfile3));
os.system('rm %s%s' % (data_dir, tmpfile4));
os.system('rm %s%s' % (data_dir, tmpfile5));
os.system('rm %s%s' % (data_dir, tmpfile1));
os.system('rm %s%s' % (data_dir, tmpfile1));